<a href="https://colab.research.google.com/github/riccardocappi/Machine_Learning_Course/blob/main/Decision_Trees_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Decision Trees implementation from scratch###
**Riccardo Cappi, 2073768**

###Create Boolean Formulas Dataset###

In [ ]:

from itertools import product
import pandas as pd
import math
import numpy as np

AND = lambda x1,x2,x3: x1 and x2 and x3
OR = lambda x1,x2,x3: x1 or x2 or x3
def XOR(x1,x2,x3):
  a = (x1 and (not x2)) or ((not x1) and x2)
  return (a and (not x3)) or ((not a) and x3)


#Fake Datasets (Boolean Formulas)
values_and = [list(a) + [AND(*a)] for a in product((False,True),repeat=3)]
values_or = [list(a) + [OR(*a)] for a in product((False,True),repeat=3)]
values_xor = [list(a) + [XOR(*a)] for a in product((False,True),repeat=3)]

dataset_and = pd.DataFrame(values_and, columns=['X1','X2','X3','Y'])
dataset_or = pd.DataFrame(values_or, columns=['X1','X2','X3','Y'])
dataset_xor = pd.DataFrame(values_xor, columns=['X1','X2','X3','Y'])

print('AND')
print(dataset_and)
print('\nOR')
print(dataset_or)
print('\nXOR')
print(dataset_xor)

AND
      X1     X2     X3      Y
0  False  False  False  False
1  False  False   True  False
2  False   True  False  False
3  False   True   True  False
4   True  False  False  False
5   True  False   True  False
6   True   True  False  False
7   True   True   True   True

OR
      X1     X2     X3      Y
0  False  False  False  False
1  False  False   True   True
2  False   True  False   True
3  False   True   True   True
4   True  False  False   True
5   True  False   True   True
6   True   True  False   True
7   True   True   True   True

XOR
      X1     X2     X3      Y
0  False  False  False  False
1  False  False   True   True
2  False   True  False   True
3  False   True   True  False
4   True  False  False   True
5   True  False   True  False
6   True   True  False  False
7   True   True   True   True


###Define Tree Object###

In [ ]:
# Define a hierarchy of classes in order to handle the construction of the Decision Tree
class Node:
  def __init__(self, label, level):
      self.label = label
      self.level = level
      

class InnerNode(Node):
  def __init__(self, label, level,information_gain):
    super(InnerNode, self).__init__(label, level)
    self.subtrees = {}
    self.information_gain = information_gain

  def __str__(self):
    s = self.label + '[ Inf. Gain: '+str(self.information_gain) + ' ]\n'
    offset = ''
    for _ in range(self.level):
      offset += '     '
    for n in self.subtrees.keys():
      s += offset + '|\n' + offset[:-1] + str(n)+'\n' + offset + '|____'  +str(self.subtrees[n])
    return s


class Leaf(Node):
  def __init__(self, label, level):
    super(Leaf, self).__init__(label, level)

  def __str__(self):
    s = str(self.label) + '\n'
    return s

###Implementation of Entropy and Information Gain###

In [ ]:
#Entropy
def E(S, classes, column_classes_label):
  entropy = 0
  for c in classes:
    card_Sc = len(S[S[column_classes_label]==c])
    pc = card_Sc / len(S)
    if pc == 1:
      return 0
    if pc == 0:
      continue
    entropy += pc*math.log2(pc)
  return -entropy

#Returns the set of values that a specific argument can have
def get_attribute_value_space(selected_attribute, S):
  values = S[[selected_attribute]].to_numpy().flatten()
  V = list(dict.fromkeys(values))
  return V

#Information Gain
def G(total_entropy, selected_attribute, S, classes, column_classes_label):
  sum = 0
  # Get set of values of 'selected_attribute'
  V = get_attribute_value_space(selected_attribute, S)
  for v in V:
    S_av = S[S[selected_attribute]==v]
    sum += (len(S_av)/len(S)) * E(S_av, classes, column_classes_label)
  return total_entropy - sum

###ID3 Implementation###

In [ ]:

def ID3(total_entropy, S, A, classes, level, column_classes_label):
  if len(A) == 0:
    majority_class = max(classes, key= lambda c: len(S[S[column_classes_label]==c]))
    return Leaf(label=majority_class, level = level)

  if total_entropy == 0:
    return Leaf(label = S[column_classes_label].values[0], level = level)

  print('Total Entropy at level '+ str(level)+': ',str(total_entropy)+ '\n')
  max_gain = np.inf
  t = tuple()
  # Find best attribute
  for a in A:
    a_gain = G(total_entropy, a, S, classes, column_classes_label)
    print(a+'[ Information Gain: ', str(a_gain) + ' ]')
    if (a_gain > max_gain) or (max_gain == np.inf):
      max_gain = a_gain
      t = (a, a_gain)
  
  best_attribute = t[0]
  T = InnerNode(label = best_attribute, level = level, information_gain = t[1])

  ### Printing information
  print('\nBest Attribute:', T)
  print('\n-----------------------------------------\n')
  ###

  A.remove(best_attribute)
  V = get_attribute_value_space(best_attribute, S)
  for v in V:
    S_v = S[S[best_attribute]==v]
    T.subtrees[v] = ID3(E(S_v,classes, column_classes_label), S_v, A.copy(), classes, level+1, column_classes_label)
  return T


def test_ID3(df, A, column_classes_label):
  classes = get_attribute_value_space(column_classes_label, df)
  tot_entropy = E(df, classes, column_classes_label)
  Tree = ID3(tot_entropy, df, A, classes,0, column_classes_label)
  print('Decision Tree representation\n')
  print(Tree)


###Test 'AND' Dataset

In [ ]:
test_ID3(dataset_and, ['X1', 'X2', 'X3'],'Y')

Total Entropy at level 0:  0.5435644431995964

X1[ Information Gain:  0.13792538097003 ]
X2[ Information Gain:  0.13792538097003 ]
X3[ Information Gain:  0.13792538097003 ]

Best Attribute: X1[ Inf. Gain: 0.13792538097003 ]


-----------------------------------------

Total Entropy at level 1:  0.8112781244591328

X2[ Information Gain:  0.31127812445913283 ]
X3[ Information Gain:  0.31127812445913283 ]

Best Attribute: X2[ Inf. Gain: 0.31127812445913283 ]


-----------------------------------------

Total Entropy at level 2:  1.0

X3[ Information Gain:  1.0 ]

Best Attribute: X3[ Inf. Gain: 1.0 ]


-----------------------------------------

Decision Tree representation

X1[ Inf. Gain: 0.13792538097003 ]
|
False
|____False
|
True
|____X2[ Inf. Gain: 0.31127812445913283 ]
     |
    False
     |____False
     |
    True
     |____X3[ Inf. Gain: 1.0 ]
          |
         False
          |____False
          |
         True
          |____True



###Test 'OR' Dataset

In [ ]:
test_ID3(dataset_or,['X1', 'X2', 'X3'],'Y')

Total Entropy at level 0:  0.5435644431995964

X1[ Information Gain:  0.13792538097003 ]
X2[ Information Gain:  0.13792538097003 ]
X3[ Information Gain:  0.13792538097003 ]

Best Attribute: X1[ Inf. Gain: 0.13792538097003 ]


-----------------------------------------

Total Entropy at level 1:  0.8112781244591328

X2[ Information Gain:  0.31127812445913283 ]
X3[ Information Gain:  0.31127812445913283 ]

Best Attribute: X2[ Inf. Gain: 0.31127812445913283 ]


-----------------------------------------

Total Entropy at level 2:  1.0

X3[ Information Gain:  1.0 ]

Best Attribute: X3[ Inf. Gain: 1.0 ]


-----------------------------------------

Decision Tree representation

X1[ Inf. Gain: 0.13792538097003 ]
|
False
|____X2[ Inf. Gain: 0.31127812445913283 ]
     |
    False
     |____X3[ Inf. Gain: 1.0 ]
          |
         False
          |____False
          |
         True
          |____True
     |
    True
     |____True
|
True
|____True



###Test 'XOR' Dataset###

In [ ]:
test_ID3(dataset_xor,['X1', 'X2', 'X3'],'Y')

Total Entropy at level 0:  1.0

X1[ Information Gain:  0.0 ]
X2[ Information Gain:  0.0 ]
X3[ Information Gain:  0.0 ]

Best Attribute: X1[ Inf. Gain: 0.0 ]


-----------------------------------------

Total Entropy at level 1:  1.0

X2[ Information Gain:  0.0 ]
X3[ Information Gain:  0.0 ]

Best Attribute: X2[ Inf. Gain: 0.0 ]


-----------------------------------------

Total Entropy at level 2:  1.0

X3[ Information Gain:  1.0 ]

Best Attribute: X3[ Inf. Gain: 1.0 ]


-----------------------------------------

Total Entropy at level 2:  1.0

X3[ Information Gain:  1.0 ]

Best Attribute: X3[ Inf. Gain: 1.0 ]


-----------------------------------------

Total Entropy at level 1:  1.0

X2[ Information Gain:  0.0 ]
X3[ Information Gain:  0.0 ]

Best Attribute: X2[ Inf. Gain: 0.0 ]


-----------------------------------------

Total Entropy at level 2:  1.0

X3[ Information Gain:  1.0 ]

Best Attribute: X3[ Inf. Gain: 1.0 ]


-----------------------------------------

Total Entropy at lev

###Test 'PlayTennis' Dataset###

In [ ]:
dataset_tennis = pd.read_csv('./tennis.csv', header=None)
dataset_tennis.columns = (['Outlook', 'Temperature', 'Humidity','Wind','PlayTennis'])
test_ID3(dataset_tennis,['Outlook', 'Temperature', 'Humidity','Wind'],'PlayTennis')

Total Entropy at level 0:  0.9402859586706309

Outlook[ Information Gain:  0.2467498197744391 ]
Temperature[ Information Gain:  0.029222565658954647 ]
Humidity[ Information Gain:  0.15183550136234136 ]
Wind[ Information Gain:  0.04812703040826927 ]

Best Attribute: Outlook[ Inf. Gain: 0.2467498197744391 ]


-----------------------------------------

Total Entropy at level 1:  0.9709505944546686

Temperature[ Information Gain:  0.5709505944546686 ]
Humidity[ Information Gain:  0.9709505944546686 ]
Wind[ Information Gain:  0.01997309402197489 ]

Best Attribute: Humidity[ Inf. Gain: 0.9709505944546686 ]


-----------------------------------------

Total Entropy at level 1:  0.9709505944546686

Temperature[ Information Gain:  0.01997309402197489 ]
Humidity[ Information Gain:  0.01997309402197489 ]
Wind[ Information Gain:  0.9709505944546686 ]

Best Attribute: Wind[ Inf. Gain: 0.9709505944546686 ]


-----------------------------------------

Decision Tree representation

Outlook[ Inf. Gain: